In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import zipfile
from sklearn.metrics import mean_squared_error
import pandas as pd
import scipy.sparse as sp
from lightfm import LightFM


c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
c:\Users\Matias\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
#pip install lightfm


In [3]:
zip_path = '../00_Data_Bases/Cluster5_1_items.zip' 
csv_filename = 'Cluster5_1_items.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df = pd.read_csv(f)

In [4]:
# Crear índices únicos para usuarios y productos
df['user_index'] = df['user_id'].astype('category').cat.codes
df['product_index'] = df['product_id'].astype('category').cat.codes

# Crear la matriz de interacciones (reordered como indicador de interacción)
interaction_matrix = sp.coo_matrix(
    (df['reordered'], (df['user_index'], df['product_index'])),
    shape=(df['user_index'].nunique(), df['product_index'].nunique())
)


In [5]:
# Características de usuarios: Frecuencia por día
user_features = sp.coo_matrix(
    pd.get_dummies(df[['user_index', 'day']], columns=['day']).groupby('user_index').sum().iloc[:, 1:].values
)

# Características de ítems: Categoría del producto
item_features = sp.coo_matrix(
    pd.get_dummies(df[['product_index', 'department']], columns=['department']).groupby('product_index').sum().iloc[:, 1:].values
)


In [6]:
interaction_matrix.data = interaction_matrix.data.astype('float32')
user_features = user_features.astype('float32')
item_features = item_features.astype('float32')


In [6]:
# Verificar la matriz de interacciones
print("Interacción - Valores no finitos:", not np.isfinite(interaction_matrix.data).all())

# Verificar user_features
print("Características de usuarios - Valores no finitos:", not np.isfinite(user_features.data).all())

# Verificar item_features
print("Características de ítems - Valores no finitos:", not np.isfinite(item_features.data).all())


Interacción - Valores no finitos: False
Características de usuarios - Valores no finitos: False
Características de ítems - Valores no finitos: False


In [7]:
model = LightFM(loss='warp', no_components=30, learning_rate=0.05)
# Reducir la matriz de interacciones
small_interaction_matrix = interaction_matrix[:1000, :50]

# Reducir las matrices de características
small_user_features = user_features[:1000, :]
small_item_features = item_features[:50, :]

# Entrenar con el subconjunto
model.fit(
    small_interaction_matrix,
    user_features=small_user_features,
    item_features=small_item_features,
    epochs=1,
    num_threads=1
)


TypeError: 'coo_matrix' object is not subscriptable

In [7]:
model = LightFM(loss='warp', no_components=30, learning_rate=0.05)

# Entrenamiento
model.fit(interaction_matrix, 
          user_features=user_features, 
          item_features=item_features, 
          epochs=1, num_threads=1)


: 

In [ ]:
print(interaction_matrix.shape)  # Dimensiones
print(interaction_matrix.nnz)    # Número de elementos no nulos


In [ ]:
print(user_features.shape)  # Debería ser (n_usuarios, n_features_usuario)
print(item_features.shape)  # Debería ser (n_items, n_features_item)
